In [1]:
with open("real_estate_sales_data.txt",encoding='utf-8') as f:
    real_estate_sales = f.read()

In [2]:
real_estate_sales

'[客户问题] 这个小区交通便利吗？\n[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。\n\n[客户问题] 我担心楼下太吵。\n[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。\n\n[客户问题] 我看房价还在涨，这个投资回报怎么样？\n[销售回答] 这个区域未来有大量的商业和基础设施建设，所以从长期来看，投资回报非常有保证。\n  \n[客户问题] 有没有学校？\n[销售回答] 附近有多所优质的学校，非常适合有孩子的家庭。\n \n[客户问题] 物业管理怎么样？\n[销售回答] 我们的物业管理得到了业主一致好评，服务非常到位。\n \n[客户问题] 我想要南向的房子。\n[销售回答] 很好，我们确实有一些朝南的单位，它们的采光特别好。\n \n[客户问题] 这个小区安全吗？\n[销售回答] 当然，我们24小时安保巡逻，还有先进的监控系统。\n \n[客户问题] 预计什么时候交房？\n[销售回答] 根据目前的进度，我们预计将在明年底交房。\n \n[客户问题] 我不想要一楼的房子。\n[销售回答] 我理解您的顾虑，我们还有多个楼层的房源可以选择。\n  \n[客户问题] 有优惠吗？\n[销售回答] 当然，如果您现在下订，我们可以给您一些优惠。\n \n[客户问题] 你们是否提供按揭服务？\n[销售回答] 是的，我们与多家银行合作，可以帮助您快速办理按揭。\n\n[客户问题] 税费怎么算？\n[销售回答] 我们可以提供详细的税费咨询服务，确保您清楚所有费用。\n\n[客户问题] 附近有医院吗？\n[销售回答] 是的，附近有多家大型医院，医疗资源非常丰富。\n\n[客户问题] 我担心小区会很拥挤。\n[销售回答] 这个小区总体规划非常合理，保证了每个单元之间有足够的空间。\n \n[客户问题] 这个小区有游泳池和健身房吗？\n[销售回答] 当然，我们提供全方位的生活设施，包括游泳池和健身房。\n \n[客户问题] 我需要两个停车位，怎么办？\n[销售回答] 我们的小区有充足的停车位，并且可以额外购买，确保您的停车需求得到满足。\n \n[客户问题] 小区里有公园吗？\n[销售回答] 是的，我们有专门设计的绿化区和儿童游乐园，确保您和家人能享受到高质量的生活。\n

In [3]:
from langchain.text_splitter import CharacterTextSplitter

In [4]:
text_splitter = CharacterTextSplitter(        
    separator = r'\n\s*\n',
    chunk_size = 80,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [5]:
docs = text_splitter.create_documents([real_estate_sales])

In [6]:
docs[0]

Document(page_content='[客户问题] 这个小区交通便利吗？\n[销售回答] 当然了，这个小区距离地铁站只有几分钟的步行距离，而且附近有多条公交线路，非常方便。')

In [7]:
len(docs)

106

In [8]:
# 使用 Faiss 作为向量数据库
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings())

C:\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [9]:
query = "小区吵不吵"

In [10]:
answer_list = db.similarity_search(query)

In [11]:
# 可以看出similarity_search可能搜出不相关的答案
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

[客户问题] 我担心楼下的商业活动会很吵。
[销售回答] 我们在规划时就已经考虑到这一点，商业区和居住区有一定的距离和隔音设计。

[客户问题] 我喜欢安静，这里噪音大吗？
[销售回答] 我们特意进行了隔音设计，并且小区内部也有绿化带，整体非常安静。

[客户问题] 我担心小区会很拥挤。
[销售回答] 这个小区总体规划非常合理，保证了每个单元之间有足够的空间。



In [12]:
db.save_local("real_estates_sale")

In [13]:
# 实例化一个 TopK Retriever
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [14]:
topK_retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x00000184E18D38D0>, search_kwargs={'k': 3})

In [15]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。

[客户问题] 我担心楼下的商业活动会很吵。
[销售回答] 我们在规划时就已经考虑到这一点，商业区和居住区有一定的距离和隔音设计。

[客户问题] 我喜欢安静，这里噪音大吗？
[销售回答] 我们特意进行了隔音设计，并且小区内部也有绿化带，整体非常安静。



In [16]:
docs = topK_retriever.get_relevant_documents("你们有没有1000万的豪宅啊？")

In [17]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 有没有可供选择的房型？
[销售回答] 我们提供多种房型，从一居室到四居室不等，满足不同家庭的需求。

[客户问题] 是否有朝南的房型？
[销售回答] 当然，我们有多款朝南的房型供您选择，享受充足的阳光和温暖。

[客户问题] 你们会提供家具吗？
[销售回答] 我们的精装房会提供基础家具和家电，让您拎包入住。



In [18]:
# 实例化一个 similarity_score_threshold Retriever
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [19]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题] 我担心楼下太吵。
[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。



In [20]:
docs = retriever.get_relevant_documents(query)

In [21]:
docs[0].page_content

'[客户问题] 我担心楼下太吵。\n[销售回答] 这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。'

In [22]:
docs[0].page_content.split("[销售回答] ")

['[客户问题] 我担心楼下太吵。\n', '这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。']

In [23]:
ans = docs[0].page_content.split("[销售回答] ")[-1]

In [24]:
ans

'这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。'

In [25]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答] ")[-1] for doc in docs]

    return ans_list

In [26]:
query = "我想离医院近点"

print(sales(query))

[]


C:\anaconda3\Lib\site-packages\langchain_core\vectorstores.py:323: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


In [27]:
# 顾客想表达的是：自己想住在小区中离医院近一些的楼栋
# 有一点点答非所问
print(sales(query, 0.75))

['有的，距离我们小区不远就有几家大型综合医院。', '附近有几所大型医院，可以满足您的各种医疗需求。', '是的，附近有多家大型医院，医疗资源非常丰富。']


In [28]:
query = "价格200万以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
# 虽然0.5有答案，但是其实答非所问
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

C:\anaconda3\Lib\site-packages\langchain_core\vectorstores.py:323: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(
C:\anaconda3\Lib\site-packages\langchain_core\vectorstores.py:323: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.8 ans: []

score:0.75 ans: []

score:0.5 ans: ['我们有不同户型和付款方案，一定有适合您预算的。', '我们提供的是市场上最有竞争力的价格，不过您如果决定购买，我们可以探讨一些优惠政策。', '我们的房子位于黄金地段，升值潜力很大，转卖起来不会有问题。', '定金金额取决于您选择的房型和付款方式，我们可以详细为您解释。']



In [29]:
# 当向量数据库中没有合适答案时，使用大语言模型
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
# RertrievalQA就是RAG，先检索，检索到了就把检索结果作为提示词给GPT-4，让GPT-4增强，检索不到就让GPT-4直接回答原问题
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

C:\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(


In [30]:
qa_chain({"query": "你们小区有200万的房子吗？"})

C:\anaconda3\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
C:\anaconda3\Lib\site-packages\langchain_core\vectorstores.py:323: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '你们小区有200万的房子吗？',
 'result': '对不起，作为一个人工智能，我没有实际的物理位置或小区，所以我无法提供有关房产的信息。我建议你直接联系房地产经纪人或者查看在线房地产网站来获取相关信息。'}

In [31]:
# 用qa_chain得到的答案更像人的语言，实现了检索+增强（“所以应该不会太吵”）
qa_chain({"query": "小区吵不吵"})

{'query': '小区吵不吵',
 'result': '这个小区特别注重居住体验，有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。所以，小区应该不会太吵。'}

In [32]:
print(sales("小区吵不吵"))

['这个小区特别注重居住体验，我们有良好的隔音设计，并且小区内部规划了绿化区域，可以有效降低噪音。']


In [33]:
# 加载 FAISS 向量数据库已有结果
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("real_estates_sale", OpenAIEmbeddings())

In [34]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4", temperature=0.5)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [35]:
qa_chain({"query": "我想买别墅，你们有么"})

C:\anaconda3\Lib\site-packages\langchain_core\vectorstores.py:323: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


{'query': '我想买别墅，你们有么', 'result': '对不起，我不能帮助您购买别墅。我是一个人工智能，我只能提供信息和回答问题。'}

In [36]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [37]:
qa_chain({"query": "我想买别墅，你们有么"})

C:\anaconda3\Lib\site-packages\langchain_core\vectorstores.py:323: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(




> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买别墅，你们有么', 'result': '对不起，我不能帮助您购买别墅，因为我是一个人工智能，没有提供房地产服务的能力。'}

In [38]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [39]:
qa_chain({"query": "我想买别墅，你们有么"})



> Entering new StuffDocumentsChain chain...


C:\anaconda3\Lib\site-packages\langchain_core\vectorstores.py:323: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(



> Finished chain.


{'query': '我想买别墅，你们有么',
 'result': '对不起，我无法提供购买别墅的服务。我是一个人工智能，主要用来提供信息和回答问题。',
 'source_documents': []}

In [40]:
# 可以看到，result综合了多个答案（source_documents)
qa_chain({"query": "周边环境怎么样"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '周边环境怎么样',
 'result': '周边环境非常好，有丰富的自然景观，包括公园和湖泊，可以提供一个宁静舒适的居住环境。同时，周围也有大型购物中心、超市、餐厅等商业设施，可以满足您的日常生活需求。还有几个公园和休闲场所，非常适合周末家庭出游。',
 'source_documents': [Document(page_content='[客户问题] 周边的自然环境如何？\n[销售回答] 周边有丰富的自然景观，包括公园和湖泊，为您提供一个宁静舒适的居住环境。'),
  Document(page_content='[客户问题] 周围的商业设施完善吗？\n[销售回答] 周围有大型购物中心、超市、餐厅等，满足您的日常生活需求。'),
  Document(page_content='[客户问题] 附近有公园或者休闲场所吗？\n[销售回答] 附近有几个公园和休闲场所，非常适合周末家庭出游。')]}